# Easily export jupyter cells to python module
https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py

In [1]:
! python /tf/main/src/scripts/notebook2script.py proc.ipynb

Converted proc.ipynb to exp/nb_proc.py


In [ ]:
#export
from pathlib import Path
from fastai.text import *

import pandas as pd
import sentencepiece as sp

import csv

In [ ]:
! pip install sklearn

In [ ]:
import sys
sys.path.append("../")
from prep.exp.nb_prep import *

In [ ]:
#export
def df_to_txt_file(df, output):
    with open(output/'text.txt', 'w') as f:
        f.write('\n'.join(list(merged_trn_df["query"]) + list(merged_trn_df["res"])))
    return output/'text.txt'

In [ ]:
#export
def gen_sp_model(df, output):
    fname = df_to_txt_file(df, output)
    sp.SentencePieceTrainer.train(f'--input={fname} --model_prefix={output}/model')

In [ ]:
#export
class SentencePiece:
    "SentencePiece wrapper."
    def __init__(self, mdl_path):
        self.mdl_path = mdl_path
        self.s = sp.SentencePieceProcessor()
        self.s.Load(str(mdl_path))
        self.vocab = [self.s.IdToPiece(id) for id in range(self.s.GetPieceSize())]
        
    def tokenize(self, inpt):
        return self.s.EncodeAsPieces(inpt)

In [ ]:
s = sp.SentencePieceProcessor()
s.Load(str(data_path/"merged/model.model"))

In [ ]:
s.SampleEncodeAsPieces??

In [ ]:
len(s)

In [ ]:
for n in range(20):
    print(len(s.EncodeAsIds('public static void adfaj')))

In [ ]:
data_path = Path("/tf/data/datasets")

In [ ]:
merged_trn_df = pd.read_csv(data_path/"merged/trn.csv")
merged_trn_df.head()

In [ ]:
df_to_txt_file(merged_trn_df, data_path/"merged")

In [ ]:
gen_sp_model(merged_trn_df, data_path/"merged")

In [ ]:
merged_trn_df.head(1).to_string(header = False, index = False)

In [ ]:
print("yo")

In [ ]:
merged_trn_df?

In [ ]:
with open(data_path/'merged/text.txt', 'w') as f:
    f.write('\n'.join(list(merged_trn_df.head(2)["query"]) + list(merged_trn_df.head(2)["res"])))

In [ ]:
len(list(merged_trn_df.head(2)["query"]) + list(merged_trn_df.head(2)["res"]))

In [ ]:
#export
def merge_cols(df):
    merged = [''.join(x) for x in zip(df["query"], df["res"])]
    new_df = pd.DataFrame({"merged": merged})
    
    return new_df

In [ ]:
#export
def conv_to_ds(df, data_path, bs = 64):
#     print(df.head(5))
    return (TextList
            .from_df(df, data_path, 
                     processor = SPProcessor(
                         sp_model = data_path/"merged/model.model",
                         sp_vocab = data_path/"merged/model.vocab"
                     ))
            .split_none()
            .label_for_lm()
            .databunch(bs = bs)
           ).train_ds

In [ ]:
#export
def gen_lm_data(df_trn, df_val, task_name, data_path, bs = 64, sample = 1):
    if task_name != "merged":
        df_trn = tag_task(df_trn, task_name)
        df_val = tag_task(df_val, task_name)
    
    df_trn = df_trn.sample(frac = sample)
    df_val = df_val.sample(frac = sample)
    
    df_trn = merge_cols(df_trn)
    df_val = merge_cols(df_val)
    
    ds_trn = conv_to_ds(df_trn, data_path)
    ds_val = conv_to_ds(df_val, data_path)
        
    data = TextLMDataBunch.create(
        train_ds = ds_trn, valid_ds = ds_val, path = data_path, bs = bs
    )

    return data

In [ ]:
df_trn, df_val, df_tst = read_data(data_path/"merged")

In [ ]:
df_trn["query"][10], df_trn["res"][10]

In [ ]:
data = gen_lm_data(df_trn, df_val, "merged", data_path, sample = 0.01)

In [ ]:
len(data.train_ds), len(data.valid_ds)

In [ ]:
data.show_batch()

# Bunch of Debugging

In [ ]:
db1 = db.dl(DatasetType.Train)

In [ ]:
 db2 = TextLMDataBunch.create(
        train_ds = db.train_dl.dl.dataset,
        valid_ds = db.train_dl.dl.dataset, path = data_path
    )

In [ ]:
db2.show_batch()

In [ ]:
db.show_batch()

In [ ]:
data.show_batch()

In [ ]:
db1 = db.databunch(bs=32)

In [ ]:
db1.show_batch()

In [ ]:
d1 = (TextList
            .from_df(df_trn[:100], data_path, 
                     processor = SPProcessor(
                         sp_model = data_path/"merged/model.model",
                         sp_vocab = data_path/"merged/model.vocab"
                     ))
            .split_none()
            .label_for_lm()
#             .databunch(bs=64)
           )

In [ ]:
d2 = d1.databunch(bs=64)
d2.show_batch()

In [ ]:
db.show_batch()

In [ ]:
data.train_dl.dl.dataset[5]

In [ ]:
new_data = (TextList
            .from_df(df_trn[:100], data_path, 
                     processor = SPProcessor(
                         sp_model = data_path/"merged/model.model",
                         sp_vocab = data_path/"merged/model.vocab"
                     ))
            .split_none()
            .label_for_lm()
            .databunch(bs=64)
           )

In [ ]:
new_data.show_batch()

In [ ]:
new_data.train_dl.dl.dataset[5]